In [25]:
import os
import torch
import pandas as pd
from torchvision import transforms
from PIL import Image
import numpy as np
from transformers import ViTForImageClassification

In [26]:
# 检查是否有可用的 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
# 加载预训练模型
# model = torch.load('../output/vit_model.pth')
# model = model.to(device)
# model.eval()

# 创建模型实例
local_model_path = "../input/pretrained_vit_model"

model = ViTForImageClassification.from_pretrained(local_model_path, num_labels=2)

# 加载状态字典
state_dict = torch.load('../output/vit_model.pth', map_location=device)

# 将状态字典加载到模型中
model.load_state_dict(state_dict)

# 将模型移动到设备（GPU或CPU）
model.to(device)

# 设置为评估模式
model.eval()

/home/wuhen/anaconda3/envs/tf/lib/python3.8/site-packages/transformers/modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [32]:
import torch
from transformers import ViTForImageClassification, ViTImageProcessor
from PIL import Image
import os

# 加载图像处理器
image_processor = ViTImageProcessor.from_pretrained(local_model_path)


def predict_images(image_folder):
    predicted_results = {}
    for image_name in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_name)
        
        # 检查文件扩展名，确保只处理图像文件
        if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        
        try:
            image = Image.open(image_path).convert("RGB")
        except (UnidentifiedImageError, IOError):
            print(f"Cannot identify image file {image_path}. Skipping.")
            continue
        
        # 调整图像大小为224x224
        image = image.resize((224, 224))
        
        inputs = image_processor(images=image, return_tensors="pt")
        
        # 将输入数据移动到设备
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # 使用模型进行推理
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)
            predicted_class = predicted.item()
                
        # 互换标签
        if predicted_class == 0:
            predicted_class = 1
        elif predicted_class == 1:
            predicted_class = 0
        
        predicted_results[image_name] = predicted_class
    return predicted_results

# 指定图片文件夹
image_folder = '../testdata'
predicted_results = predict_images(image_folder)

In [33]:
# 将结果写入output.csv
import pandas as pd
import os

# 去掉文件格式后缀并对字典的键（文件名）进行排序
sorted_predicted_results = sorted(
    (os.path.splitext(image_name)[0], predicted_class) for image_name, predicted_class in predicted_results.items()
)

# 将排序后的结果转换为DataFrame
df = pd.DataFrame(sorted_predicted_results, columns=['ImageName', 'PredictedClass'])

# 将结果写入output.csv
df.to_csv('../cla_pre.csv', index=False, header=False)
print('ok')

ok


In [34]:
# 预测结果的含义
label_map = {
    0: "real",
    1: "ai"
}
# 将预测结果转换为可读的标签
readable_results = {image_name: label_map[predicted_class] for image_name, predicted_class in predicted_results.items()}

# 打印可读的预测结果
for image_name, predicted_label in readable_results.items():
    print(f"{image_name}: {predicted_label}")

0lhPySoNr8rvwHsH.jpeg: real
QKBP3o4s040osbUw.jpg: ai
1dD6BWKuGIO7EAWz.jpeg: real
4wb7IhftslpoePra.jpeg: real
HzfAL3uWeroASH9o.jpg: ai
0UUOxjrYlX9UHTcU.jpg: ai
oN5xbpCPpn5MDfBJ.jpg: real
s3BeVJ0PR9mOA3lE.jpg: ai
s4RlE63lgpYnk2zr.jpg: ai
nXajkXU8XAy25rDB.jpg: ai
HwCv0N9NWinSaiQo.jpg: ai
6oVER8Pbh1ODZpRC.jpeg: real
sbO4xkL2HTmrDW3V.jpg: ai
AjXrDnjnIk9jE0sd.jpg: ai
a5LgI7uQC3dImBu9.jpg: real
j9QOCTl9R3r3BX0k.jpg: ai
PYnOLsR0UA4Eta4A.jpg: ai
6iByTZavgNTirhvS.jpeg: real
NzS612gGF8ymBSaZ.jpg: ai
6AEXujYvvn6RVaiT.jpeg: real
784TfPmhrjs0vvH9.jpg: ai
9bbyaPt6hrjXbND9.jpg: ai
BhkVVDN4kbYI6N0O.jpg: real
3qTRJf4l4ef9EYlN.jpeg: real
1UTWSUcIjR8Ca1G4.jpeg: real
Jhte5Q4Zv2XoZNpT.jpg: ai
CHx29ePPJxnhvkqp.jpg: ai
JaKGyl1YyRDpvNaL.jpg: ai
azSFTkU33EgfHbB2.jpg: ai
FzRBs7JGvACw0EnI.jpg: ai
0xk0nXzqhz2tG3wY.jpeg: real
0A4bl3w75vOKQJCa.jpeg: real
kDiOxre02h7B0y9w.jpg: ai
4jRbmBDwM72mLFlH.jpeg: real
7JC1iB5LLGwGg8bE.jpg: ai
q1uJ51IWg9CVgtX8.jpg: ai
bUGOsBAslkiVppX5.jpg: real
6FYiojLWA1ElcCl2.jpeg: real
4pjiSt